In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
from google.colab.patches import cv2_imshow
from random import shuffle 

In [ ]:
def getLabels(typ):
  if typ == 0: #open
    return [1,0]
  else:        #close
    return [0,1]

In [ ]:
from skimage.transform import resize  # resizing images


items1 = os.listdir("/content/drive/My Drive/Datasets/open_eye")
items2 = os.listdir("/content/drive/My Drive/Datasets/close_eye")

i = 0

IMG_SIZE = 24
x_data = np.empty((len(items1)+len(items2), IMG_SIZE, IMG_SIZE, 1), dtype=np.float32)
y_data = np.empty((len(items1)+len(items2),2),dtype=np.float32)


for each_image in items1:
  
  full_path = "/content/drive/My Drive/Datasets/open_eye/" + each_image
  image = cv2.imread(full_path)
  #print(image.shape)
  img_cvt = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
  img = resize(img_cvt, output_shape=(IMG_SIZE, IMG_SIZE, 1), preserve_range=True)
  x_data[i] = img
  y_data[i] = [1.0,0.0]

  i += 1
  #print(img_cvt.shape)
  #cv2_imshow(image)
  #print(image)

len(items1)
#len(items2)

672


In [ ]:
print(i)

672


In [ ]:
full_path = "/content/B0.jpg"
img = cv2.imread(full_path)
res = cv2.resize(img, dsize=(24, 24), interpolation=cv2.INTER_CUBIC)
cv2_imshow(res)
res.shape

(24, 24, 3)

In [ ]:
from google.colab.patches import cv2_imshow
cv2_imshow(x_data[0])

In [ ]:
print(len(items1))
print(len(items2))
print(i)

672
698
672


In [ ]:


for each_image in items2:
  full_path = "/content/drive/My Drive/Datasets/close_eye/" + each_image
  image = cv2.imread(full_path)
  img_cvt = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
  img = resize(img_cvt, output_shape=(IMG_SIZE, IMG_SIZE, 1), preserve_range=True)
  x_data[i] = img
  y_data[i] = [0.0,1.0]

  i += 1

#x_data.shape

In [ ]:
from keras.models import Sequential 
from keras.layers import Dropout,Conv2D,Flatten,Dense, MaxPooling2D, BatchNormalization 
from keras.layers.normalization import BatchNormalization 
from keras.utils import np_utils

In [ ]:


x_data = x_data/255
x_data[0].shape

(24, 24, 1)

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split( x_data, y_data, test_size=0.25, random_state=4)
print ('Train set:', x_train.shape,  y_train.shape)
print ('Test set:', x_test.shape,  y_test.shape)

Train set: (1027, 24, 24, 1) (1027, 2)
Test set: (343, 24, 24, 1) (343, 2)


In [ ]:
model=Sequential()

model.add(Conv2D(32,kernel_size=(3, 3), activation='relu', input_shape=(IMG_SIZE,IMG_SIZE,1)))
model.add(MaxPooling2D(pool_size=(1,1)))

model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(1,1)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(1,1)))

model.add(Dropout(0.25))
model.add(Flatten())

model.add(Dense(128,activation='relu'))

model.add(Dropout(0.50))

model.add(Dense(2, activation='softmax'))

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),batch_size=20,epochs=30,verbose=1)

Epoch 1/30
52/52 [==============================] - 5s 90ms/step - loss: 0.4376 - accuracy: 0.7668 - val_loss: 0.2039 - val_accuracy: 0.9679
Epoch 2/30
52/52 [==============================] - 4s 80ms/step - loss: 0.1161 - accuracy: 0.9723 - val_loss: 0.1868 - val_accuracy: 0.9708
Epoch 3/30
52/52 [==============================] - 4s 80ms/step - loss: 0.0563 - accuracy: 0.9874 - val_loss: 0.1315 - val_accuracy: 0.9738
Epoch 4/30
52/52 [==============================] - 4s 81ms/step - loss: 0.0393 - accuracy: 0.9937 - val_loss: 0.1378 - val_accuracy: 0.9854
Epoch 5/30
52/52 [==============================] - 4s 80ms/step - loss: 0.0158 - accuracy: 0.9971 - val_loss: 0.1023 - val_accuracy: 0.9796
Epoch 6/30
52/52 [==============================] - 4s 81ms/step - loss: 0.0159 - accuracy: 0.9982 - val_loss: 0.0985 - val_accuracy: 0.9883
Epoch 7/30
52/52 [==============================] - 4s 81ms/step - loss: 0.0256 - accuracy: 0.9966 - val_loss: 0.1378 - val_accuracy: 0.9913
Epoch 8/30
52

In [ ]:
import time
t = time.time()

export_path_keras = "/content/drive/MyDrive/Datasets/cnnEye.h5".format(int(t))
print(export_path_keras)

model.save(export_path_keras)

/content/drive/MyDrive/Datasets/cnnEye.h5


In [ ]:
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 22, 22, 32)        320       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 22, 22, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 20, 20, 32)        9248      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 20, 20, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 18, 18, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 18, 18, 64)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 18, 18, 64)       